#### Gaze Inference
Add inference about gaze distributions to EduSense video pipeline. 

In [ ]:
! pip install sshtunnel
from sshtunnel import SSHTunnelForwarder
import pymongo
import urllib
import pprint
import base64
from io import BytesIO

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import json

Define server constants

In [4]:
MONGO_HOST = ""
MONGO_DB = ""
MONGO_USER = ""
MONGO_PASS = ""
VIDEO_ID = ""

In [ ]:
print("Connecting to MongoDB...")

server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017) # change to 3000/3001
)
server.start()
# local port connection

try:
    client = pymongo.MongoClient('127.0.0.1', server.local_bind_port)
    db = client[MONGO_DB]
    collection = db[VIDEO_ID]
    print("\033[92mConnection successful\033[0m")
except: 
    print("Error connecting to database")

#### Instructor gaze inference

In [ ]:
def getInstructorGaze(collection, subject = "student", return_errors = False):

    gaze_df = pd.DataFrame(columns=['sx', 'sy', 'tx', 'ty', 'fr', 'thumb'])
    num_errors = 0
    gaze_vec = []
    
    for frame in collection:
        if frame["channel"] == "student":
            frame_num = frame["frameNumber"]
            item["thumbnail"]['binary']
            for person in frame["people"]:
                try:
                    gaze_vec = person["inference"]["head"]["gazeVector"]
                    angle = (person["inference"]["head"]["yaw"])
                    source = person["inference"]["head"]["gazeVector"]
                    
                    x1, y1 = source[0]
                    line_length = 100
                    (y2,x2) = (y1 + line_length*math.cos(math.radians(angle)),x1 + line_length*math.sin(math.radians(angle))*-1)
                    y2, x2 = y2, x2
                    gaze_vec = [[x1, y1], [x2, y2]]
                    
                except:
                    num_errors += 1

                if(len(gaze_vec) == 2):
                    temp_vec = pd.DataFrame([np.append(np.append((1/16)*np.array(gaze_vec[0]), 
                                                                 (1/16)*np.array(gaze_vec[1])), frame_num)], 
                                            columns=['sx', 'sy', 'tx', 'ty', 'fr'])
                    gaze_df.loc[len(gaze_df)] = temp_vec.loc[0]

                else: 
                    num_errors += 1
    if(return_errors):
        return num_errors
    return gaze_df

In [ ]:
def orthog(x) :
    y = np.empty_like(x)
    y[0] = -x[1]
    y[1] = x[0]
    return(y)

def vec_intersect(x_1,x_2,y_1,y_2) :
    dap = orthog(x_2-x_1)
    denom = np.dot( dap, (y_2-y_1))
    num = np.dot( dap, (x_1-y_1) )
    return ((num / denom.astype(float))*(y_2-y_1)+y_1)

def bucketGaze(v, bounds, dist = False):
    b_sx = np.array( [v.sx, v.sy] )
    b_sy = np.array( [v.tx, v.ty] )
    b_tx = np.array( [0, 80] )
    b_ty = np.array( [300, 80] )
    intersect_pt = vec_intersect( b_sx,b_sy, b_tx,b_ty )
    
    if dist 
      return(intersect_pt[0])

    else: 
      bucket = 'C'
      if(intersect_pt[0] < bounds[0]):
          bucket = 'L'
      elif(intersect_pt[0] < bounds[1]):
          bucket = 'C'
      else:
          bucket = 'R'
      return(bucket)

In [ ]:
db_coll = collection.find({"people.0.inference": {"$exists": True}})

stu_gaze_data = getGazeData(db_coll, 
                            subject="student", 
                            return_errors=False)

ins_gaze_data = getGazeData(db_coll, 
                            subject="instructor", 
                            return_errors=False)

#### Student gaze inference

In [ ]:
def studentHeadYaw(collection):
  yaws = []
  indices = []

  col = collection.find({"people.0.inference": {"$exists": True}})
  for item in col:
      if item["channel"] == "student":
          t_yaw = 0
          for person in item["people"]:
              t_yaw += person["inference"]["head"]["yaw"]
              
          t_yaw = t_yaw / len(item["people"])
          
          # Print interesting images w/ yaw vector
          if t_yaw < -10:
              test = item["thumbnail"]
          
              img_data = base64.b64encode(test['binary'])
              with open(str(item["frameNumber"]) + ".jpeg", "wb") as fh:
                  fh.write(base64.decodebytes(img_data))

              im = Image.open(str(item["frameNumber"]) + ".jpeg")
              
              draw = ImageDraw.Draw(im)
              print(t_yaw)
              draw.line((im.size[0]/2, im.size[1]/2, im.size[0]/2 - (im.size[1]/2)*math.tan(math.radians(temp_yaw)), im.size[1]), fill=100, width=4)
              im.show()
              im.save("test-im.png")
              
          yaws.append(t_yaw)
          indices.append(item["frameNumber"])
          
  fig = plt.figure()
  plt.plot(yaws, indices)
  plt.xlabel('Degree Head Rotation (Yaw)')
  plt.ylabel('Time --->')

In [ ]:
def studentHeadPitch(collection):
  pitch = []
  indices = []

  col = collection.find({"people.0.inference": {"$exists": True}})
  for item in col:
      if item["channel"] == "student":
          temp_p = 0
          for person in item["people"]:
              temp_p += person["inference"]["head"]["pitch"]
              
          temp_p = temp_p / len(item["people"])
              
          pitch.append(temp_p)
          indices.append(item["frameNumber"])
          
  fig = plt.figure()
  plt.plot(indices, pitch)
  plt.xlabel('Time --->')
  plt.ylabel('Degree Head Rotation (Up/Down)')